In [121]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [122]:
# Download training data
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [123]:
batch_size = 64

# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [124]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [125]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [126]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [127]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()

    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss = loss_fn(pred, y).item()
            correct += (pred.argmax(dim=1) == y).type(torch.float).sum().item()
        
    test_loss /= num_batches
    correct /= size

    print(f"Test error:\n\tAccuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}")

In [128]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n---------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
---------------------------------
loss: 2.309588  [   64/60000]
loss: 2.293212  [ 6464/60000]
loss: 2.270949  [12864/60000]
loss: 2.263487  [19264/60000]
loss: 2.240457  [25664/60000]
loss: 2.217494  [32064/60000]
loss: 2.230232  [38464/60000]
loss: 2.195473  [44864/60000]
loss: 2.197083  [51264/60000]
loss: 2.166054  [57664/60000]
Test error:
	Accuracy: 47.8%, Avg loss: 0.013790
Epoch 2
---------------------------------
loss: 2.167205  [   64/60000]
loss: 2.156587  [ 6464/60000]
loss: 2.092558  [12864/60000]
loss: 2.115104  [19264/60000]
loss: 2.053104  [25664/60000]
loss: 1.995687  [32064/60000]
loss: 2.041749  [38464/60000]
loss: 1.951917  [44864/60000]
loss: 1.970835  [51264/60000]
loss: 1.906772  [57664/60000]
Test error:
	Accuracy: 53.6%, Avg loss: 0.012190
Epoch 3
---------------------------------
loss: 1.925564  [   64/60000]
loss: 1.897625  [ 6464/60000]
loss: 1.768363  [12864/60000]
loss: 1.821610  [19264/60000]
loss: 1.699319  [25664/60000]
loss: 1.649927  [32064/600

In [129]:
torch.save(model.state_dict(), "model.pth")
print("Saved model to model.pth")

Saved model to model.pth


In [130]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

In [131]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
